In [ ]:
!pip install nuscenes-devkit &> /dev/null  # Install nuScenes.
!pip install dicttoxml
!mkdir /maps
!mkdir save_files

mkdir: cannot create directory ‘/maps’: File exists


In [ ]:
import json
from xml.dom.minidom import parseString
import xml.etree.ElementTree as ET
from nuscenes.map_expansion.map_api import NuScenesMap
from nuscenes.map_expansion import arcline_path_utils
from collections import defaultdict
import numpy as np

In [ ]:
!git clone https://github.com/argoai/argoverse-api.git
!pip install -e argoverse-api/
!apt-get install vtk6
!apt-get install libvtk6-dev python-vtk6
!pip install mayavi
!pip install PyQt5

fatal: destination path 'argoverse-api' already exists and is not an empty directory.
Obtaining file:///content/argoverse-api
  Cloning https://github.com/daavoo/pyntcloud to /tmp/pip-install-cilj7b23/pyntcloud
  Running command git clone -q https://github.com/daavoo/pyntcloud /tmp/pip-install-cilj7b23/pyntcloud
     |████████████████████████████████| 1.5MB 8.1MB/s 
  Created wheel for pyntcloud: filename=pyntcloud-0.1.3-cp36-none-any.whl size=346176 sha256=b016d2a19135ae72669c1ae0c3c7b10ad547ed3d6f482f05d6c5871f12d2ac22
  Stored in directory: /tmp/pip-ephem-wheel-cache-f5lff800/wheels/be/37/76/afc097c015aa7cdb2ccfc317e1ea8684c387d6b48984dd1ded
  Created wheel for lap: filename=lap-0.4.0-cp36-cp36m-linux_x86_64.whl size=1589020 sha256=2b91624fbc7f9b34afb6b9ac6e72e0bb3081e2dc54b2d31f4ce87e7e5901d6ef
  Stored in directory: /root/.cache/pip/wheels/da/3e/af/eddcd6ffaa27df8d0ddac573758f8953c4e57c64c4c8c8b7d0
Successfully built pyntcloud lap
  Running setup.py develop for argoverse
Reading p

Upload the json files for the nuscenes maps to the session before running the next cel

In [ ]:
from argoverse.utils.manhattan_search import compute_point_cloud_bbox

## Id to assign to lanes in the new xml map file
global_way_id = 8000000
# Load json file. Loop over all the json files in directory
for filename in ['boston-seaport','singapore-hollandvillage','singapore-onenorth','singapore-queenstown']:
    print(filename)
    with open(f"/maps/{filename}.json") as f:
        data = json.load(f)
    nusc_map = NuScenesMap(dataroot='/', map_name=filename)

    root = ET.Element('NuScenesMap')
    # Create new xml ETree

    # Map lane token of nuscenes to node_tokens that form the centerline 
    lane_dict = defaultdict(list) # k: token, v: list of node_token

    # Mao x,y value of node to the new node id in the xml
    present_nodes = {} # k:tuple, v: node_id
    global_id = 0 # New id for nodes in the xml

    # Loop over all lanes in the json to populate the lane to node dictionary
    for way in data['lane']:
        if way['token'] == '8f23d3ed-1089-4fcf-a178-a174abc69938':
          continue
        lane_record = nusc_map.get_lane(way['token']) # get arcline associated with lane
        poses = arcline_path_utils.discretize_lane(lane_record, resolution_meters=0.5) # discretize the lane to given resolution
        for pose in poses:
            currNode = (pose[0], pose[1])
            if currNode not in present_nodes:
                node = ET.SubElement(root, 'node')
                node.set('id', str(global_id))
                present_nodes[currNode] = global_id
                global_id += 1
                node.set('x', str(pose[0]))
                node.set('y', str(pose[1]))
            lane_dict[way['token']].append(present_nodes[currNode])

    # Map nodes in the original json to their x, y coordinates
    node_dict = {} # k: node_token v: (x,y)

    # Loop over all nodes in the json to populate the node_dict
    for node in data['node']:
        node_dict[node['token']] = (node['x'], node['y'])

    # Map polygons in the original json to the x,y coordinates of their exterior nodes
    poly_dict = {} # k: poly_token v: np.ndarray of shape (N,2)

    # Loop over all nodes in the json to populate the node_dict
    for poly in data['polygon']:
        poly_array = []
        for node_id in poly['exterior_node_tokens']:
          poly_array.append(list(node_dict[node_id]))
        poly_dict[poly['token']] = np.array(poly_array)
      
    # Map that links new lane_id in the xml to its original token in the json
    way_to_lane_id = {} # k: global_way_id , v: data['lane']['token']

    # map lane segment IDs to their index in the table
    tableidx_to_laneid_map = {}
    # array that holds xmin,ymin,xmax,ymax for each coord
    halluc_bbox_table = []
    table_idx_counter = 0

    ## Iterate over the lanes to create the required xml and supporting files
    for way in data['lane']:
        if way['token'] == '8f23d3ed-1089-4fcf-a178-a174abc69938':
            continue
        node = ET.SubElement(root, 'way')

        if way['token'] not in way_to_lane_id:
          way_to_lane_id[way['token']] = global_way_id
          global_way_id+=1
        curr_id = way_to_lane_id[way['token']]
          

        node.set('lane_id',str(curr_id))
        traffic = ET.SubElement(node, 'tag')
        traffic.set('k', "has_traffic_control")
        traffic.set('v', "False")

        turn = ET.SubElement(node, 'tag')
        turn.set('k', "turn_direction")
        turn.set('v', "NONE")

        intersection = ET.SubElement(node, 'tag')
        intersection.set('k', "is_intersection")
        intersection.set('v', "False")

        ln = ET.SubElement(node, 'tag')
        ln.set('k', "l_neighbor_id")
        ln.set('v', "None")

        rn = ET.SubElement(node, 'tag')
        rn.set('k', "r_neighbor_id")
        rn.set('v', "None")



        for waypoint in lane_dict[way['token']]:
            nd = ET.SubElement(node, 'nd')
            nd.set('ref', str(waypoint))

        predecessors = nusc_map.get_incoming_lane_ids(way['token'])
        successors = nusc_map.get_outgoing_lane_ids(way['token'])

        for pred_id in predecessors:
            pre = ET.SubElement(node, 'tag')
            pre.set('k', "predecessor")
            if pred_id not in way_to_lane_id:
              way_to_lane_id[pred_id] = global_way_id
              global_way_id+=1
            int_pred_id = way_to_lane_id[pred_id]
            pre.set('v', str(int_pred_id))

        for succ_id in successors:
            succ = ET.SubElement(node, 'tag')
            succ.set('k', "successor")
            if succ_id not in way_to_lane_id:
              way_to_lane_id[succ_id] = global_way_id
              global_way_id+=1
            int_succ_id = way_to_lane_id[succ_id]
            succ.set('v', str(int_succ_id))


        lane_id = way_to_lane_id[way['token']]
        tableidx_to_laneid_map[table_idx_counter] = lane_id
        table_idx_counter += 1
        # if you dont have the lane polygon boundaries, hallucinate it
        # halluc_lane_polygon = centerline_to_polygon(lane_centerline)
        # otherwise use the lane polygon directly
        xmin, ymin, xmax, ymax = compute_point_cloud_bbox(poly_dict[way['polygon_token']])
        halluc_bbox_table += [(xmin,ymin,xmax,ymax)]

    halluc_bbox_table = np.array(halluc_bbox_table)
    halluc_bbox_dict = {
      "tableidx_to_laneid_map": tableidx_to_laneid_map,
      "halluc_bbox_table": halluc_bbox_table
    }
    np.save(f"save_files/{filename}_halluc_bbox_table.npy", halluc_bbox_table)
    with open(f"save_files/{filename}_tableidx_to_laneid_map.json", 'w') as outfile:
        json.dump(tableidx_to_laneid_map, outfile)
        
    tree = ET.ElementTree(root)
    with open (f"save_files/{filename}.xml", "wb") as files : 
            tree.write(files) 


boston-seaport
singapore-hollandvillage
singapore-onenorth
singapore-queenstown


In [ ]:
!unzip drive-download-20201217T094305Z-001.zip

Archive:  drive-download-20201217T094305Z-001.zip
   creating: drive-download-20201217T094305Z-001/
  inflating: drive-download-20201217T094305Z-001/pruned_nuscenes_BSP_10318_vector_map.xml  
  inflating: drive-download-20201217T094305Z-001/pruned_nuscenes_SHV_10320_vector_map.xml  
  inflating: drive-download-20201217T094305Z-001/pruned_nuscenes_SON_10322_vector_map.xml  
  inflating: drive-download-20201217T094305Z-001/pruned_nuscenes_SQT_10324_vector_map.xml  


In [ ]:
from argoverse.utils.manhattan_search import compute_point_cloud_bbox
from argoverse.utils.centerline_utils import centerline_to_polygon

In [ ]:
for filename in ['pruned_nuscenes_BSP_10318_vector_map','pruned_nuscenes_SHV_10320_vector_map','pruned_nuscenes_SHV_10320_vector_map','pruned_nuscenes_SHV_10320_vector_map']:
  with open(f"/drive-download-20201217T094305Z-001/{filename}.xml") as f:
      data = ET.parse(f)
      root = data.getroot()


  